In [1]:
#Muticlass Classification

#Muticlass Classification is a classification task which has 'one target variable' 
#and that target variable can be more than 2 classes 

In [2]:
import warnings
warnings.warn = lambda *a, **kw: False


from scipy.io import loadmat
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
mnist_raw = loadmat("mnist-original.mat")
X, Y = mnist_raw['data'].T, mnist_raw['label'].T
x_train, y_train, x_test, y_test = X[:60000], Y[:60000], X[60000:], Y[60000:]

shuffled_index = np.random.permutation(60000)
x_train, y_train = x_train[shuffled_index], y_train[shuffled_index]
some_digit = X[36000]

In [3]:
#some algorithms are capable of handlingmuticlass classification directly (eg. RandomForest, Bayes classifier)
#but some algoritms(e.g. SGDClassification) do not have that capability (they support only binary classification)
#So, there are 2 ways to create muticlass classifier
#1. OvA: create binary classifiers equal to the number of classes. each clf responsible for classifing each classes
#2. OvO: create binary classifiers equal to the number of each pairs of class. each clf responsible for ditinguish between that pair of classes

In [4]:
#create muticlassifier using SGDClassifier
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_train, y_train)

print("prediction of some_digit is", sgd_clf.predict([some_digit]))


#you can look at the decision score of each classifier
#and the highest score is corresponding to class 5
print ("score for each classes are ")
print (sgd_clf.decision_function([some_digit]))

prediction of some_digit is [5.]
score for each classes are 
[[-113127.42918324 -500345.07500293 -296767.6902453   -96996.52231522
  -619870.34413645  -19161.26819285 -794235.7539054  -381700.26168843
  -711143.73098082 -485967.84129898]]


In [9]:
#try OvO method
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(x_train, y_train)

print ("prediction of some_digit")
ovo_clf.predict([some_digit])
print ("number of classifier", len(ovo_clf.estimators_))

prediction of some_digit
number of classifier 45
decision scores are


array([[7.26692917, 2.89703187, 3.9651707 , 8.33291076, 4.90291258,
        9.5       , 0.61072833, 3.94081462, 1.75760668, 1.82589529]])

In [12]:
#try RandomForest which can handle muticlass directly
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(x_train, y_train)

forest_clf.predict([some_digit])

array([5.])

In [13]:
forest_clf.predict_proba([some_digit])

array([[0. , 0. , 0. , 0. , 0. , 0.9, 0. , 0.1, 0. , 0. ]])

In [15]:
#next, try to evaluate model performance using cross-validation
from sklearn.model_selection import cross_val_score
sgd_accuracy_scores = cross_val_score(SGDClassifier(random_state=42), x_train, y_train, cv=3, scoring='accuracy')
sgd_accuracy_scores
#you can use accuracy as measurement metric because the number of instances for each classes are balance

array([0.87227554, 0.86719336, 0.88293244])

In [17]:
#improve SGDClassifer using Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float64))
sgd_scaledfeature_accuracy_scores = cross_val_score(SGDClassifier(random_state=42), x_train_scaled, y_train, cv=3, scoring='accuracy')
sgd_scaledfeature_accuracy_scores

array([0.90866827, 0.90744537, 0.91038656])

In [ ]:
#the results are better because of feature scaling